# Assignment #1: Performance Benchmarking of Cryptographic Mechanisms



Trabalho por: 
- David Rodrigues up202303949
- Diana Pereira up202304476
- Simão Gomes up202304752

### Index
[0. Libreries and Modules](#importing-required-libraries-and-modules) <br>
[A. Generating text files](#a-generating-random-text-files) <br>
[B. AES](#b-encrypting-and-decrypting-using-aes)<br>
- [B.1. Results](#b1-comparing-aes-results)<br>
- [B.2. Padding](#b2-padding-vs-no-padding)<br>
[C. RSA]

### Importing required libraries and modules

In [18]:
import os
import timeit

### A. Generating random text files

Files with the following sizes:
- Advanced Encription Standard (in bytes): 8, 64, 512, 4096, 32768, 262144, 2097152
- Rivest-Shamir-Adleman (in bytes): 8, 64, 512, 4096, 32768, 262144, 2097152
- Secure Hash Algorithm (in bytes): 2, 4, 8, 16, 32, 64, 128

In [13]:
tamanhosAES = [8,64,512,4096,32768,262144,2097152]
tamanhosRSA = [8,64,512,4096,32768,262144,2097152]
tamanhosSHA = [2,4,8,16,32,64,128]

os.makedirs("Ficheiros",exist_ok=True) #exist_ok para resolver o erro de já existir o diretório "FileExistsError"

def gerar_ficheiros(tamanho,nome):
    with open(f"Ficheiros/{nome}", "wb") as file:
        file.write(os.urandom(tamanho))

for t in tamanhosAES:
    gerar_ficheiros(t, f"AES_{t}.txt")

for t in tamanhosRSA:
    gerar_ficheiros(t, f"RSA_{t}.txt")

for t in tamanhosSHA:
    gerar_ficheiros(t, f"SHA_{t}.txt")

### B. Encrypting and decrypting using AES
To get statistically significant results we will repeat the encription->decription procedure several times for each file <br>
We will be using padding  

In [19]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.backends import default_backend


In [27]:
key_AES=os.urandom(32) #key de 256 bits
iv_AES=os.urandom(16) #iv de 128 bits

def encrypt_AES(original):
    cipher = Cipher(algorithms.AES(key_AES), modes.CBC(iv_AES))
    encryptor = cipher.encryptor()

    #padding para tornar o texto múltiplo de 16 bytes
    padder = padding.PKCS7(128).padder()
    padded = padder.update(original) + padder.finalize()

    return encryptor.update(padded) + encryptor.finalize()

def decrypt_AES(encrypted):
    cipher = Cipher(algorithms.AES(key_AES), modes.CBC(iv_AES))
    decryptor = cipher.decryptor()

    decrypted= decryptor.update(encrypted) + decryptor.finalize()

    unpadder = padding.PKCS7(128).unpadder()

    return unpadder.update(decrypted) + unpadder.finalize()

tempos_enc_AES = []
tempos_dec_AES = []
reps = 100

for t in tamanhosAES:
    with open(f"Ficheiros/AES_{t}.txt","rb") as file:
        original = file.read()

    tempo_enc = 0
    tempo_dec = 0
    
    for i in range(reps):
        start = timeit.default_timer()
        encrypted = encrypt_AES(original)
        end = timeit.default_timer()
        tempo_enc += end - start

        start = timeit.default_timer()
        decrypt_AES(encrypted)
        end = timeit.default_timer()
        tempo_dec += end - start

    tempo_enc /= reps
    tempo_dec /= reps

    tempo_enc *= 1e6 #converte para microsegundos
    tempo_dec *= 1e6

    tempos_enc_AES.append(tempo_enc)
    tempos_dec_AES.append(tempo_dec)

    print(t ,"bytes:")
    print(f"Encryption time: {tempo_enc: .5f} microsegundos")
    print(f"Decryption time: {tempo_dec: .5f} microsegundos")
    print("")


8 bytes:
Encryption time:  130.21100 microsegundos
Decryption time:  124.02800 microsegundos

64 bytes:
Encryption time:  86.08000 microsegundos
Decryption time:  86.32400 microsegundos

512 bytes:
Encryption time:  134.44300 microsegundos
Decryption time:  130.89800 microsegundos

4096 bytes:
Encryption time:  125.07400 microsegundos
Decryption time:  115.26900 microsegundos

32768 bytes:
Encryption time:  195.57600 microsegundos
Decryption time:  138.64400 microsegundos

262144 bytes:
Encryption time:  865.40700 microsegundos
Decryption time:  408.32700 microsegundos

2097152 bytes:
Encryption time:  13350.93200 microsegundos
Decryption time:  8674.94800 microsegundos



#### B.1. Comparing AES Results

#### B.2. Padding vs No Padding

### C. Encrypting and decrypting with RSA

In [16]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import rsa, padding

In [17]:
private_key = rsa.generate_private_key(public_exponent=65537,key_size=2048,backend=default_backend())
public_key =private_key.public_key()

def encrypt_RSA(original):
    encrypted = public_key.encrypt(
        original, 
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return encrypted

def decrypt_RSA(encrypted):
    decrypted = private_key.decrypt(
        encrypted, 
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return decrypted

resultados_RSA = []
reps = 100

for t in tamanhosRSA:
    with open(f"Ficheiros/RSA_{t}.txt","rb") as file:
        original = file.read()

    tempo_enc = 0
    tempo_dec = 0
    
    for i in range(reps):
        start = timeit.default_timer()
        encrypted = encrypt_RSA(original)
        end = timeit.default_timer()
        tempo_enc += end - start

        start = timeit.default_timer()
        decrypt_RSA(encrypted)
        end = timeit.default_timer()
        tempo_dec += end - start

    tempo_enc /= reps
    tempo_dec /= reps

    tempo_enc *= 1e6 #converte para microsegundos
    tempo_dec *= 1e6

    resultados_RSA.append((t, tempo_enc, tempo_dec))

    print(t , f"bytes - encryption time:{tempo_enc: .2f} microssegundos and decryption time:{tempo_dec: .2f} microsegundos")

8 bytes - encryption time: 146.63 microssegundos and decryption time: 1752.23 microsegundos
64 bytes - encryption time: 111.06 microssegundos and decryption time: 1401.26 microsegundos


ValueError: Encryption failed